In [15]:
!pip install langchain
!pip install huggingface_hub
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 41.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 59.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.1 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125925 sha256=03f1a791c82300ad91185ef93fcbbb5cf8358298918d02f8fead6faf5ebaeb06
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [16]:
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"] = ""

In [20]:
from langchain.document_loaders import TextLoader
loader = TextLoader('sample.txt')
documents = loader.load()

In [21]:
documents

[Document(page_content='"The President shall from time to time give to Congress information of the State of the Union and recommend to their Consideration such measures as he shall judge necessary and expedient." Article II, Sec. 3, U.S. Constitution\n\nGeorge Washington rode on a carriage driven by six horses from his house on Cherry Street to Federal Hall in New York to give his deliver his first annual message in person.\nOn a cold January morning, the President rode in a carriage drawn by six horses from his residence on Cherry Street in New York to Federal Hall for a joint meeting of the two bodies of Congress, the House of Representatives and the Senate. When George Washington personally delivered the first annual message to Congress on January 8, 1790, he was aware of his constitutional duty to deliver his message and of the precedent he was setting for future presidents.\n\nThe President\'s focus, however, was on the very concept of union itself. Washington and his administrati

In [22]:
import textwrap

def wrap_text_preserve_newlines(text, width=110):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

In [23]:
print(wrap_text_preserve_newlines(str(documents[0])))

page_content='"The President shall from time to time give to Congress information of the State of the Union
and recommend to their Consideration such measures as he shall judge necessary and expedient." Article II,
Sec. 3, U.S. Constitution\n\nGeorge Washington rode on a carriage driven by six horses from his house on
Cherry Street to Federal Hall in New York to give his deliver his first annual message in person.\nOn a cold
January morning, the President rode in a carriage drawn by six horses from his residence on Cherry Street in
New York to Federal Hall for a joint meeting of the two bodies of Congress, the House of Representatives and
the Senate. When George Washington personally delivered the first annual message to Congress on January 8,
1790, he was aware of his constitutional duty to deliver his message and of the precedent he was setting for
future presidents.\n\nThe President\'s focus, however, was on the very concept of union itself. Washington and
his administration were co

In [30]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

In [31]:
len(docs)

5

In [32]:
# Embeddings
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings()

In [33]:
!pip install faiss-cpu

In [34]:
from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, embeddings)

In [41]:
query = "who was first President to revive Washington's spoken precedent?"
docs = db.similarity_search(query)

In [42]:
print(wrap_text_preserve_newlines(str(docs[0].page_content)))

For example, Thomas Jefferson thought Washington's oral presentation was too kingly for the new republic.
Likewise, Congress's practice of giving a courteous reply in person at the President's residence was too
formal. Jefferson detailed his priorities in his first annual message in 1801 and sent copies of the written
message to each house of Congress. The President's annual message, as it was then called, was not spoken by
the President for the next 112 years. The message was often printed in full or as excerpts in newspapers for
the American public to read.

The first President to revive Washington's spoken precedent was Woodrow Wilson in 1913. Although controversial
at the time, Wilson delivered his first annual message in person to both houses of Congress and outlined his
legislative priorities.


In [37]:
from langchain.chains.question_answering import load_qa_chain
from langchain import HuggingFaceHub

In [38]:
llm=HuggingFaceHub(repo_id="google/flan-t5-xl", model_kwargs={"temperature":0, "max_length":512})

In [39]:
chain = load_qa_chain(llm, chain_type="stuff")

In [ ]:
query = ""
docs = db.similarity_search(query)
chain.run(input_documents=docs, question=query)